In [45]:
!pip install Algebra

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 30.0 MB/s eta 0:00:00
  Created wheel for Algebra: filename=algebra-1.2.1-py3-none-any.whl size=21077 sha256=888ecdecf276bcfdec9ecc4307bde17ded06cbb1959311550e8b6180c178e12b
  Stored in directory: c:\users\pkitt\appd

In [2]:
!pip show Algebra
!which python  # or !where python on Windows

Name: algebra
Version: 1.2.1
Summary: Algebraic structures
Home-page: https://github.com/wesselb/algebra
Author: Wessel Bruinsma
Author-email: wessel.p.bruinsma@gmail.com
License: MIT
Location: C:\Users\pkitt\anaconda3\Lib\site-packages
Requires: backends, numpy, plum-dispatch
Required-by: 


'which' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!git clone https://github.com/rkycia/KHNN.git

fatal: destination path 'KHNN' already exists and is not an empty directory.


In [4]:
import sys
sys.path.append("C:/Users/pkitt/KHNN")  # Adjust path if needed

In [5]:
import Algebra
from Hyperdense import HyperDense

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, activations, initializers
import Algebra

In [7]:
class HyperDense(layers.Layer):
    """Dense layer when the weight are calculated based on arbitrary hyperalgebra.
    The usage is similar to standard hyperdense layer. The input of data is 
    batch x (dim_algebra x n), where n is arbitrary nonzero integer, i.e. the last 
    axis has dimension that is an integer multiple of algebra dimension.
    
    The class is a generalization of class described in:
    G. Vieira and M. Eduardo Valle, "Acute Lymphoblastic Leukemia Detection Using Hypercomplex-Valued Convolutional Neural Networks," 2022 International Joint Conference on Neural Networks (IJCNN), Padua, Italy, 2022, pp. 1-8, doi: 10.1109/IJCNN55064.2022.9892036.
    The calulations are adjusted to TensorFlow tensor operations, and are enhanced to 
    arbitrary algebras.
    """
    def __init__(self, units,
                 activation=None,
                 use_bias=True,
                 kernel_initializer=tf.keras.initializers.GlorotNormal(seed=0),
                 bias_initializer='zeros',
                 algebra = Algebra.Quaternions # Quaternion algebra
                ):
        """The input is the same as for Dense layer. The additional parametr is 
        algebra which is set do algebra.Quaternions.
        """
        super(HyperDense, self).__init__()
        self.units = int(units) if not isinstance(units, int) else units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.A = tf.convert_to_tensor(algebra.getA(), dtype = np.float32)
        self.algebra_dim = algebra.getDim()

In [8]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [9]:
df.cat =df.select_dtypes(include=[object])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_13888\2763109048.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [10]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_13888\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
le = LabelEncoder()

In [13]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [14]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [15]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [16]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,8,4
1,4,4,4
2,4,4,2
3,4,0,0
4,3,6,5


In [17]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [18]:
import tensorflow as tf

In [19]:
from tensorflow import keras

In [20]:
X = df_train[["Humidity","Moisture","Soil Type","Crop Type"]]

In [21]:
y = df_train[["Fertilizer Name"]]

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [48]:
from collections import defaultdict
import random

def class_based_reservoir_sampling(stream, k):
    """
    Maintain separate reservoirs per class label.
    :param stream: iterable of (sample, label) tuples
    :param k: buffer size per class
    :return: dict of class-wise reservoirs
    """
    reservoirs = defaultdict(list)
    class_counts = defaultdict(int)  # counts per class for reservoir logic

    for sample, label in stream:
        count = class_counts[label]
        buffer = reservoirs[label]

        if count < k:
            buffer.append(sample)
        else:
            j = random.randint(0, count)
            if j < k:
                buffer[j] = sample

        class_counts[label] += 1

    return reservoirs
reservoirs = class_based_reservoir_sampling(zip(X, y), k=50)

In [49]:
import numpy as np
print(np.unique(y, return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6]), array([113887, 114436, 112453, 110889, 111158,  94860,  92317],
      dtype=int64))


In [50]:
#Define model:
from tensorflow.keras.models import Sequential
model = Sequential()
def build_model():
    input_dim = 16
    num_neurons = 4
    return input_dim + num_neurons
    model.add(HyperDense(num_neurons))

In [51]:
 #model.add(Dense(num_neurons)) #real numbers alternative forcomparision
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
model.add(Activation('tanh'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [52]:
 #Setup learning
from tensorflow.keras.optimizers import Adam
opt = Adam()
model.compile(loss='binary_crossentropy', optimizer=opt,
metrics=['accuracy'])

In [53]:
 #Train model
model.fit(X, y, epochs=5, verbose=0)

In [54]:
history = model.fit(X, y, epochs=5, verbose=0)

In [55]:
print(history.history)

{'accuracy': [0.1525813341140747, 0.1525813341140747, 0.1525813341140747, 0.1525813341140747, 0.1525813341140747], 'loss': [-1085.3629150390625, -1283.270263671875, -1481.4617919921875, -1679.16357421875, -1877.317138671875]}


In [56]:
import numpy as np
print(np.unique(y, return_counts=True))

(array([0, 1, 2, 3, 4, 5, 6]), array([113887, 114436, 112453, 110889, 111158,  94860,  92317],
      dtype=int64))


In [32]:
print(model.output_shape)

(None, 1)


In [83]:
 #Make prediction
y_predict = model.predict(X_train, verbose=0)
y_predict_quantized = np.round(y_predict).astype(int)

In [84]:
 import torch
 import torch.nn as nn
 from collections import OrderedDict
 import matplotlib.pylab as plt
 from HyperdenseTorch import HyperDenseTorch

In [126]:
#Define model:
model = nn.Sequential(OrderedDict([
 ("HyperDense", HyperDenseTorch(10, (4,), activation =
 torch.tanh )),
 ("Dense", nn.Linear(40,7)),
 ('Sigmoid', nn.Sigmoid())
 ]))

In [127]:
#Setup learning
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.015)
torch.manual_seed(1)
num_epoch = 5
loss_hist_train = [0]*num_epoch
accuracy_hist_train = [0]*num_epoch
loss_hist_train = [0]*num_epoch

In [129]:
# Prepare input & target
X_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_tensor = torch.tensor(y_train.values, dtype=torch.long).squeeze()  # shape: [N]

# KHNN model should output raw logits of shape [batch_size, 7]
# Loss function should be CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

for epoch in range(num_epoch):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    logits = model(X_tensor)  # shape: [N, 7]
    
    # Compute loss
    loss = loss_fn(logits, y_tensor)
    loss.backward()
    optimizer.step()
    
    # Store loss
    loss_hist_train[epoch] = loss.item()
    
    # Compute accuracy
    predictions = torch.argmax(logits, dim=1)              # class indices
    accuracy = (predictions == y_tensor).float().mean()
    accuracy_hist_train[epoch] = accuracy.item()

In [130]:
accuracy_hist_train[epoch] = accuracy
print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}, Accuracy = {accuracy:.4f}")

Epoch 5: Loss = 1.9478, Accuracy = 0.1517


In [131]:
print("X_tensor:", X_tensor.shape, X_tensor.dtype)
print("y_tensor:", y_tensor.shape, y_tensor.dtype)
print("Model output:", model(X_tensor).shape)
print("Sample logits:", model(X_tensor)[0])
print("Unique targets:", torch.unique(y_tensor))

X_tensor: torch.Size([502500, 4]) torch.float32
y_tensor: torch.Size([502500]) torch.int64
Model output: torch.Size([502500, 7])
Sample logits: tensor([0.6611, 0.5322, 0.3596, 0.5144, 0.4207, 0.3670, 0.5135],
       grad_fn=<SelectBackward0>)
Unique targets: tensor([0, 1, 2, 3, 4, 5, 6])
